In [ ]:
!aws s3 cp --recursive s3://app-hkaa/output/hkaa-object-dection-new-2019-04-25-07-45-00/ output/hkaa-object-dection-new-2019-04-25-07-45-00/ 
!aws s3 cp --recursive s3://app-hkaa/source-images/ source-images/

In [2]:
import time
import json
import cv2
import random
import numpy as np
from deep_sort import preprocessing
from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
import generate_detections as gdet

In [3]:
base_dir = 'source-images/'
detections_base_dir = 'output/hkaa-object-dection-new-2019-04-25-07-45-00/'
file_names = os.listdir(base_dir)
jpg_file_names = []
for file_name in file_names:
    if file_name.endswith('.jpg'):
        jpg_file_names.append(int(file_name.replace('frame', '').replace('.jpg', '')))
jpg_file_names = sorted(jpg_file_names)
jpg_file_names = ['frame'+str(file_name)+'.jpg' for file_name in jpg_file_names]

In [4]:
colors = dict()

def visualize_track(img_file, dets, classes=[], thresh=0.6):
    """
    visualize track in one image
    Parameters:
    ----------
    img : numpy.array
        image, in bgr format
    dets : numpy.array
        ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
        each row is one object
    classes : tuple or list of str
        class names
    thresh : float
        score threshold
    """    

    img=cv2.imread(img_file)
    global colors
    
    # x,y,w,h  filter the detection confidence >= thresh.  
    boxs = [[int(d[2]*width),int(d[3]*height),int((d[4]-d[2])*width),int((d[5]-d[3])*height)] for d in dets if d[1]>=thresh]

    features = encoder(img,boxs)

    # score to 1.0 here).
    detections = [Detection(bbox, 1.0, feature) for bbox, feature in zip(boxs, features)]
    # Run non-maxima suppression.
    boxes = np.array([d.tlwh for d in detections])
    scores = np.array([d.confidence for d in detections])
    indices = preprocessing.non_max_suppression(boxes, nms_max_overlap, scores)
    detections = [detections[i] for i in indices]
    # Call the tracker
    tracker.predict()
    tracker.update(detections)
    
    # visualize the confirmed track
    for track in tracker.tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue 
        bbox = track.to_tlbr()
        track_id = int(track.track_id)
        if track_id not in colors:
            colors[track_id] = (int(random.random() * 255), int(random.random() * 255), int(random.random() * 255))
        cv2.rectangle(img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),colors[track_id], thickness=3)
        cv2.putText(img, str(track.track_id), (int(bbox[0]), int(bbox[1]) - 2), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=2, thickness=3, color=(255, 255, 255))
    return img

In [ ]:
# save_video
fps = 30
size = (1280, 720)
videoWriter = cv2.VideoWriter('vidoeTrack.mp4', cv2.VideoWriter_fourcc('H', '2', '6', '4'), fps, size)
cnt = 0
start = time.time()

# Definition of the parameters
max_cosine_distance = 0.3
nn_budget = None
nms_max_overlap = 1.0

# deep_sort 
model_filename = 'model/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename,batch_size=1)

metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

height = 720
width = 1280

for file_name in jpg_file_names:
    if cnt % 100 == 0:
        print(cnt, '/', len(jpg_file_names), time.time()-start)
    cnt += 1

    detections_file_name = detections_base_dir+file_name+'.out'
    file_name = base_dir + file_name

    detections = json.load(open(detections_file_name, 'r'))
    object_categories = ['hammer', 'truck']
    # Setting a threshold 0.20 will only plot detection results that have a confidence score greater than 0.20.
    threshold = 0.5

    # Visualize the tracks.
    try:
        frame = visualize_track(file_name, detections['prediction'], object_categories, threshold)
    except Exception as e:
        print(e)
    videoWriter.write(frame)
videoWriter.release()

0 / 7228 0.11783218383789062
